In [1]:
import pandas as pd
import numpy as np
import requests

## Get song list

In [2]:
# load dataset - songs & artist
df = pd.read_csv('merge_w_lyrics_full.csv')[['title', 'artist_x', 'artist_y']]
df

,title,artist_x,artist_y
0,(I Called Her) Tennessee,Tim Dugger,NaN
1,"10,000 Reasons (Bless the Lord)",Matt Redman,NaN
2,100 Proof,Kellie Pickler,NaN
3,101,Alicia Keys,NaN
4,110%,Jessie Ware,NaN
...,...,...,...
21310,Danny Phantom,Trippie Redd,Trippie Redd Featuring XXXTENTACION
21311,What's Wrong,Rod Wave,Rod Wave
21312,Demon Time,Trippie Redd,Trippie Redd Featuring Ski Mask The Slump God
21313,Tick Tock,Young Thug,Young Thug


In [7]:
track_id = [np.nan] * len(df)
date = [np.nan] * len(df)

## Song index range of each year

In [4]:
import os

filename = [f for f in os.listdir('song_release')]
filename.remove('.DS_Store')
filename = sorted([f for f in filename if int(f[:4]) in range(2012, 2022)])

year_idx = {}
counter = 0
for i in range(0, len(filename)):
    year_idx[filename[i][:4]] = range(counter, counter + len(pd.read_csv('song_release/'+filename[i])))
    counter += len(pd.read_csv('song_release/'+filename[i]))

year_idx

{'2012': range(0, 2620),
 '2013': range(2620, 5517),
 '2014': range(5517, 8415),
 '2015': range(8415, 11035),
 '2016': range(11035, 13118),
 '2017': range(13118, 15146),
 '2018': range(15146, 16723),
 '2019': range(16723, 18107),
 '2020': range(18107, 19381),
 '2021': range(19381, 20027)}

## Search for *track_id* and *release_date*

In [229]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

for i in range(df.loc[~df['track_id'].isnull()].index.values + 1, len(df)):
    # limit year
    try:
        year = [y for y in year_idx if i in year_idx[y]][0]
    except:
        year = '2012-2021'
    # search query
    if pd.isna(sub['artist_y'][i]):
        query = 'track:'+df['title'][i]+'%20artist:'+df['artist_x'][i]+'%20year:'+year+'&type=track&limit=50'
    else:
        query = 'track:'+df['title'][i]+'%20artist:'+df['artist_y'][i]+'%20year:'+year+'&type=track&limit=50'
    # actual GET request with proper header
    r = requests.get(BASE_URL + 'search?q=' + query, headers=headers)
    
    # traverse items returned to match title and artist
    try:
        for item in r.json()['tracks']['items']: 
            artist = item['artists'][0]['name']
            if (artist in df['artist_x'][i]) or (artist in df['artist_y'][i]):
                track_id[i] = item['id']
                date[i] = item['album']['release_date']
                print(i, sub['track_id'][i])
                break
    except:
        pass

df['track_id'] = track_id
df['release_date'] = date

/var/folders/lv/snv97l992c5ctbbxj96wt76w0000gn/T/ipykernel_12303/1991702567.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['track_id'][i] = item['id']
/var/folders/lv/snv97l992c5ctbbxj96wt76w0000gn/T/ipykernel_12303/1991702567.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['release_data'][i] = item['album']['release_date']


5359 4zJVt5xo6CgtQ7QGdvvDuL
5381 1LeWh0PHKHYNfxi358UcTi
